In [1]:
from music21 import *

In [9]:
A = corpus.getComposer('bach')

In [12]:
X = corpus.parse(A[0])

In [18]:
X.metadata.title

'bwv1.6.mxl'

In [23]:
type(A)

music21.stream.Score

In [35]:
a = AP[0].flat.getElementsByClass(note.Note)

In [95]:
mynotes = A.parts[0].flat.getElementsByClass(note.Note)
crd = A.chordify().flat.getElementsByClass(chord.Chord)
bass = crd[-1].bass().pitchClass

In [96]:
all_notes = [x for x in mynotes]
all_notes_transposed = [(x.pitchClass-bass)%12 for x in mynotes]

In [5]:
from hmmlearn import hmm

In [153]:
def scale_degree_parser(unparsed):
    counter = 0
    sd_dict = {}
    hmm_notes = []
    lengths = []
    all_parsed = []
    for x in unparsed:
        all_parsed.append(converter.parse(x))
    for x in all_parsed:
        if type(x) is stream.Score:
            A = x
            for part in range(len(A.parts)):
                mynotes = A.parts[part].flat.getElementsByClass(note.Note)
                crd = A.chordify().flat.getElementsByClass(chord.Chord)
                bass = crd[-1].bass().pitchClass
                all_notes_transposed = [(j.pitchClass-bass)%12 for j in mynotes]
                lengths.append(len(all_notes_transposed))
                for n in all_notes_transposed:
                    if n in sd_dict.keys():
                        hmm_notes.append([sd_dict[n]])
                    elif n not in sd_dict.keys():
                        
                        hmm_notes.append([counter])
                        counter += 1
                        sd_dict[n] = counter
                
        elif type(x) is stream.Opus:
            for A in x:
                for part in range(len(A.parts)):
                    mynotes = A.parts[part].flat.getElementsByClass(note.Note)
                    crd = A.chordify().flat.getElementsByClass(chord.Chord)
                    bass = crd[-1].bass().pitchClass
                    all_notes_transposed = [(j.pitchClass-bass)%12 for j in mynotes]
                    lengths.append(len(all_notes_transposed))
                    for n in all_notes_transposed:
                        if n in sd_dict.keys():
                            hmm_notes.append([sd_dict[n]])
                        elif n not in sd_dict.keys():
                            
                            hmm_notes.append([counter])
                            counter += 1
                            sd_dict[n] = counter
    print(hmm_notes)
    print(lengths)
    Z = hmm.MultinomialHMM(n_components=3, verbose=False,n_iter=1000,tol=.01)
    FIT_DATA = Z.fit(hmm_notes, lengths)
    tokens = []
    for x in range(len(sd_dict)):
        for a, b in sd_dict.items():
            if x == b:
                tokens.append(a)
    return {'fit':FIT_DATA, 'tokens':tokens}

In [154]:
FIT_DATA = scale_degree_parser(corpus.getComposer('palestrina')[0:1])

[[0], [1], [2], [2], [3], [3], [4], [2], [4], [5], [2], [4], [3], [4], [2], [2], [5], [2], [4], [3], [6], [7], [3], [4], [2], [2], [5], [7], [1], [1], [8], [2], [9], [5], [2], [6], [2], [5], [2], [2], [2], [6], [3], [6], [2], [1], [5], [2], [2], [4], [7], [4], [3], [7], [1], [7], [3], [7], [2], [2], [9], [5], [5], [2], [2], [6], [2], [2], [4], [3], [1], [3], [4], [4], [3], [7], [1], [1], [7], [7], [3], [7], [1], [1], [1], [9], [5], [2], [1], [1], [9], [5], [2], [4], [2], [2], [4], [3], [3], [7], [1], [1], [1], [8], [5], [8], [1], [1], [4], [1], [1], [5], [9], [5], [8], [1], [7], [2], [1], [8], [1], [2], [5], [7], [7], [7], [1], [3], [7], [1], [1], [1], [7], [3], [4], [4], [9], [1], [10], [7], [4], [3], [7], [1], [5], [1], [7], [7], [1], [7], [3], [7], [1], [9], [5], [5], [4], [2], [5], [9], [1], [7], [7], [7], [2], [7], [1], [1], [9], [5], [2], [5], [5], [2], [4], [2], [5], [8], [1], [2], [6], [2], [3], [2], [4], [1], [1], [7], [3], [4], [4], [2], [3], [1], [9], [5], [2], [5], [8], [1]

/home/daniel/anaconda3/lib/python3.6/site-packages/hmmlearn/hmm.py:405: RuntimeWarning: divide by zero encountered in log
  return np.log(self.emissionprob_)[:, np.concatenate(X)].T


In [155]:
emiss = FIT_DATA['fit'].emissionprob_
tokens = FIT_DATA['tokens']
trans = FIT_DATA['fit'].transmat_
print(emiss)

[[  0.00000000e+000   2.47950352e-001   1.03632341e-001   1.90014638e-001
    2.07254930e-001   5.14552316e-003   6.90311124e-018   2.41683181e-001
    6.69290930e-066   2.14940150e-049   4.31903459e-003]
 [  7.02521749e-003   2.41269231e-001   1.10313706e-001   8.01418542e-015
    1.08819658e-001   4.69345229e-001   6.32269574e-002   7.86507336e-010
    2.34678800e-106   7.09867988e-059   5.66455900e-102]
 [  0.00000000e+000   5.20975989e-002   4.76581224e-001   4.31655650e-002
    3.76456055e-003   2.29314034e-021   7.97186090e-070   4.34316307e-002
    1.00630790e-001   2.80328631e-001   4.27948232e-033]]


In [156]:
import matplotlib.pyplot as plt
import os
import matplotlib as mpl
mpl.rcParams['patch.force_edgecolor'] = True
mpl.rcParams['patch.facecolor'] = 'b'


def pie_nodes(fit_data_dict):
    emiss = fit_data_dict['fit'].emissionprob_
    tokens = fit_data_dict['tokens']
#     K = len(emiss)
    plt.close()
    K = 3
    for k in range(K):
        plt.figure(figsize=(2,2), tight_layout=True)
        emit1_data = []
        emit1_labels = []
        plt.rcParams['font.size'] = 14.0
        plt.rcParams['font.family'] = 'serif'
        plt.axis('equal')
        # csfont = {'fontname':'Comic Sans MS'}
        kcolors = ['white']
        other_numb = 0
        for a, b in zip(emiss[k], tokens):
            if a >= .03:
                emit1_data.append(a)
                emit1_labels.append(b)
            else:
                other_numb += a
        emit1_data.append(other_numb)
        emit1_labels.append('other')

        a, b = plt.pie(emit1_data, labels=emit1_labels, radius=2, colors=kcolors, labeldistance=0.8)
        for bb in b:
            bb.set_horizontalalignment('center')
            bb.set_fontsize(12)
        pltpath = '/home/daniel/Desktop'
        if not os.path.exists(pltpath):
            os.makedirs(pltpath)
        plt.savefig(pltpath+'/%s_%s.png' %(K, k), bbox_inches='tight')
        plt.close()

In [157]:
pie_nodes(FIT_DATA)

/home/daniel/anaconda3/lib/python3.6/site-packages/matplotlib/figure.py:1742: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


In [158]:
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, to_agraph
def neato_function(K):
    import networkx as nx
    from networkx.drawing.nx_agraph import write_dot, to_agraph
    import pygraphviz as pgv
    G = nx.DiGraph()
    labels =[x+1 for x in range(K)]
    for x, y in zip(trans, labels):
        for a, b in zip(x, labels):
            G.add_edge(y, b, penwidth=a*8)
    edge_weights = []
    edges = G.edges(data=True)
    print('edges:', edges)
    for x in edges:
        edge_weights.append(x[2]['penwidth'])
    nodes = G.nodes()
    print('nodes:', nodes)
    for x in range(len(nodes)):
        G.node[nodes[x]]['shape'] = 'none'
        G.node[nodes[x]]['image'] = '/home/daniel/Desktop/%s_%s.svg' %(K,x)
        G.node[nodes[x]]['fontcolor'] = 'blue'
        G.node[nodes[x]]['fontsize'] = 40
    G = to_agraph(G)
    # G=pgv.AGraph(ranksep=10)
    G.draw('/home/daniel/Desktop/function.pdf', format='pdf', prog='dot')

In [159]:
neato_function(3)

edges: [(1, 1, {'penwidth': 6.6035619374540691}), (1, 2, {'penwidth': 0.0054456532530323329}), (1, 3, {'penwidth': 1.390992409292898}), (2, 1, {'penwidth': 0.79913059403427145}), (2, 2, {'penwidth': 1.5828712153150279}), (2, 3, {'penwidth': 5.6179981906507006}), (3, 1, {'penwidth': 1.671504362503512}), (3, 2, {'penwidth': 6.3261418720549365}), (3, 3, {'penwidth': 0.0023537654415526528})]
nodes: [1, 2, 3]


TypeError: cannot use a string pattern on a bytes-like object

0.99999999999999989

In [ ]:
def function_graph_maker(corpus, corpus_name, mode, K_range, path):
    X = corpus_function
    for K in K_range:
        if not os.path.exists('%s/%s_%s_%s.pkl' % (path, corpus_name, mode, K)):
            Y = hmm.MultinomialHMM(n_components=K, verbose=True,n_iter=1000,tol=.001)
            Y.n_features=len(X[2])
            FIT_DATA = Y.fit(X[0], X[1])
            TOKEN_NAMES=X[2]
            pickleit = {'fit': FIT_DATA, 'tokens': TOKEN_NAMES}
            if not os.path.exists(path):
                os.makedirs(path)
            joblib.dump(pickleit, '%s/%s_%s_%s.pkl' %(path, corpus_name, mode, K), compress=9)
        
        FIT_DATA = joblib.load('%s/%s_%s_%s.pkl' %(path, corpus_name, mode, K))
        emiss = FIT_DATA['fit'].emissionprob_
        tokens = FIT_DATA['tokens']
        trans = FIT_DATA['fit'].transmat_

def pie_nodes(fit_data_dict):
    emiss = fit_data_dict['fit'].emissionprob_
    tokens = fit_data_dict['tokens']
#     K = len(emiss)
    plt.close()
    for k in range(K):
        plt.figure(figsize=(2,2), tight_layout=True)
        emit1_data = []
        emit1_labels = []
        plt.rcParams['font.size'] = 14.0
        plt.rcParams['font.family'] = 'serif'
        plt.axis('equal')
        # csfont = {'fontname':'Comic Sans MS'}
        kcolors = ['white']
        other_numb = 0
        for a, b in zip(emiss[k], tokens):
            if a >= .03:
                emit1_data.append(a)
                emit1_labels.append(b)
            else:
                other_numb += a
        emit1_data.append(other_numb)
        emit1_labels.append('other')

        a, b = plt.pie(emit1_data, labels=emit1_labels, radius=2, colors=kcolors, labeldistance=0.8)
        for bb in b:
            bb.set_horizontalalignment('center')
            bb.set_fontsize(12)
        pltpath = '%s/%s/%s' %(path, corpus_name, mode)
        if not os.path.exists(pltpath):
            os.makedirs(pltpath)
        plt.savefig(pltpath+'/%s_%s.png' %(K, k), bbox_inches='tight')
        plt.close()
pie_nodes(FIT_DATA)

def neato_function(K):
    import networkx as nx
    from networkx.drawing.nx_agraph import write_dot, to_agraph
    import pygraphviz as pgv
    G = nx.DiGraph()
    labels =[x+1 for x in range(K)]
    for x, y in zip(trans, labels):
        for a, b in zip(x, labels):
            if a >= .01: #threshold*******
                G.add_edge(y, b, penwidth=a*8, weight=int(a*100))
    edge_weights = []
    edges = G.edges(data=True)
#             print('edges:', edges)
    for x in edges:
        edge_weights.append(x[2]['penwidth'])
    nodes = G.nodes()
#             print('nodes:', nodes)
    for x in range(len(nodes)):
        G.node[nodes[x]]['shape'] = 'none'
#                 G.node[nodes[x]]['image'] = '/home/daniel/Desktop/54.svg'
        G.node[nodes[x]]['image'] = '%s/%s/%s/%s_%s.png' %(path, corpus_name, mode, K,x)
        print('%s/%s/%s/%s_%s.svg' %(path, corpus_name, mode, K,x))
        G.node[nodes[x]]['fontcolor'] = 'blue'
        G.node[nodes[x]]['fontsize'] = 40
    G = to_agraph(G)
    # G=pgv.AGraph(ranksep=10)
    if not os.path.exists(path):
        os.makedirs(path)
    print(G)
#             G.draw('tryit.pdf', format='pdf', prog='dot')
    G.draw('%s/%s_%s_%s.pdf' %(path, corpus_name, mode, K), format='pdf', prog='dot')
neato_function(K)
print('finished', corpus_name, mode, K)